## Import packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [2]:
# Fixed variables
RANDOM_STATE = 12

# Path variables
train_csv_path = "/kaggle/input/house-prices-advanced-regression-techniques/train.csv"
test_csv_path = "/kaggle/input/house-prices-advanced-regression-techniques/test.csv"

## Data preprocessing

In [3]:
def preprocess_dataset(path):
    data = pd.read_csv(path)
    drop_columns = ['Id', 'Alley', 'PoolQC', 'Fence', 'MiscFeature']
    data = data.drop(columns=drop_columns)
    return data

In [4]:
df = preprocess_dataset(train_csv_path)
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000


## Partitioning the dataset

In [5]:
X = df.drop(columns=['SalePrice'])
Y = df['SalePrice']

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=RANDOM_STATE)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((1168, 75), (292, 75), (1168,), (292,))

## Model training

In [7]:
# lr_model = LinearRegression().fit(X_train, Y_train)
# predictions = lr_model.predict(X_test)

In [8]:
from sklearn.metrics import r2_score